In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

DB Password: ········


In [2]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [3]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [4]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    try:
        Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    except:
        print("DataFrame is empty. Condition does not fit to any record!")
        raise SystemExit(0)
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        for mrn in cohort_indexes: 
            x = get_mrn_has_certain_condition_WINDOW(mrn,cohort_condition,df_mrn.loc[mrn]["HT_Onset"]-gap_in_days)
            a = [mrn, x]
            cohort_mrn_diagnosis.append(a)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [5]:
case = df_to_cohort(Case_filtered_15_540)
control = df_to_cohort(Control_filtered_15_540)

In [7]:
Med_schizo_condition = (Drug('%Aripiprazole%') | 
                            Drug('%Quetiapine %'))


In [8]:
##### CASES ######

In [9]:
Med_schizo_EVER = get_has_certain_condition(Med_schizo_condition, Case_filtered_15_540, "Med_schizo", 180 , "EVER", "Case")

Fetching data for Drug (...)


In [11]:
Med_schizo_EVER.loc[Med_schizo_EVER['has_condition_Med_schizo'] == 1]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_schizo
medical_record_number,,,,,,,,,,,,,
1784990016,74,31540.0,26671.0,26904.0,26671.0,Age_ICD_condition,25677,994.0,1930,Other,Jehovah's Witness,Female,1.0
182340809,961,25986.0,27091.0,25988.0,25986.0,Age_BP_condition,25009,977.0,1933,Unknown,PT Declined,Female,1.0
1863831568,196,28309.0,28309.0,NaN,28309.0,Age_BP_condition,25184,3125.0,1932,White,Jewish,Male,1.0
1908380284,250,NaN,NaN,15944.0,15944.0,Age_MED_condition,14944,1000.0,1970,Other,None,Female,1.0
2354349508,401,16001.0,NaN,19313.0,16001.0,Age_BP_condition,13368,2633.0,1964,Unknown,Baptist,Male,1.0
2385936406,3017,17495.0,NaN,17023.0,17023.0,Age_MED_condition,14297,2726.0,1961,African-American,Pt Declined,Male,1.0
2700958543,179,27183.0,NaN,27441.0,27183.0,Age_BP_condition,25013,2170.0,1942,White,Unknown,Female,1.0
3136949252,138,NaN,15644.0,15483.0,15483.0,Age_MED_condition,14530,953.0,1971,White,Catholic,Male,1.0
3805325741,36,16721.0,NaN,NaN,16721.0,Age_BP_condition,16131,590.0,1972,Other,Catholic,Male,1.0


In [12]:
Med_schizo_COUNT = get_has_certain_condition(Med_schizo_condition, Case_filtered_15_540, "Med_schizo", 180 , "COUNT", "Case")

Fetching data for Drug (...)


In [13]:
Med_schizo_COUNT.loc[Med_schizo_COUNT['number_of_occurences_Med_schizo'] != 0]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_Med_schizo
medical_record_number,,,,,,,,,,,,,
1784990016,74,31540.0,26671.0,26904.0,26671.0,Age_ICD_condition,25677,994.0,1930,Other,Jehovah's Witness,Female,1.0
182340809,961,25986.0,27091.0,25988.0,25986.0,Age_BP_condition,25009,977.0,1933,Unknown,PT Declined,Female,1.0
1863831568,196,28309.0,28309.0,NaN,28309.0,Age_BP_condition,25184,3125.0,1932,White,Jewish,Male,1.0
1908380284,250,NaN,NaN,15944.0,15944.0,Age_MED_condition,14944,1000.0,1970,Other,None,Female,1.0
2354349508,401,16001.0,NaN,19313.0,16001.0,Age_BP_condition,13368,2633.0,1964,Unknown,Baptist,Male,1.0
2385936406,3017,17495.0,NaN,17023.0,17023.0,Age_MED_condition,14297,2726.0,1961,African-American,Pt Declined,Male,1.0
2700958543,179,27183.0,NaN,27441.0,27183.0,Age_BP_condition,25013,2170.0,1942,White,Unknown,Female,1.0
3136949252,138,NaN,15644.0,15483.0,15483.0,Age_MED_condition,14530,953.0,1971,White,Catholic,Male,2.0
3805325741,36,16721.0,NaN,NaN,16721.0,Age_BP_condition,16131,590.0,1972,Other,Catholic,Male,2.0


In [14]:
Med_schizo = Med_schizo_EVER.merge(Med_schizo_COUNT['number_of_occurences_Med_schizo'], left_index= True, right_index = True)

In [15]:
Med_schizo

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_schizo,number_of_occurences_Med_schizo
medical_record_number,,,,,,,,,,,,,,
100067499,82,24103.0,NaN,24106.0,24103.0,Age_BP_condition,21049,3054.0,1951,White,Other,Male,0.0,0.0
100091035,154,11369.0,12938.0,12881.0,11369.0,Age_BP_condition,9141,2228.0,1978,Ba,Other,Male,0.0,0.0
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,0.0,0.0
100423963,20,NaN,23584.0,23584.0,23584.0,Age_ICD_condition,19827,3757.0,1945,Other,Pt Declined,Male,0.0,0.0
1005188482,279,20636.0,20514.0,20514.0,20514.0,Age_ICD_condition,19884,630.0,1958,Ba,Jewish,Male,0.0,0.0
1006954100,28,15242.0,15536.0,15536.0,15242.0,Age_BP_condition,13828,1414.0,1973,Unknown,Catholic,Female,0.0,0.0
1006987597,72,27618.0,27618.0,27618.0,27618.0,Age_BP_condition,26706,912.0,1941,White,Catholic,Male,0.0,0.0
1007749798,93,NaN,27233.0,27233.0,27233.0,Age_ICD_condition,25235,1998.0,1939,White,Catholic,Female,0.0,0.0
1007977434,221,25355.0,24347.0,24347.0,24347.0,Age_ICD_condition,23605,742.0,1943,White,Unknown,Male,0.0,0.0


In [16]:
Med_schizo.to_parquet('Med_schizo_Case.parquet')

In [ ]:
##### CONTROLS######

In [17]:
Med_schizo_EVER_Control = get_has_certain_condition(Med_schizo_condition, Control_filtered_15_540, "Med_schizo", 180 , "EVER", "Control")


Fetching data for Drug (...)


In [18]:
Med_schizo_EVER_Control.loc[Med_schizo_EVER_Control['has_condition_Med_schizo'] == 1]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_schizo
medical_record_number,,,,,,,,,
1000165222,17506,627,12215,5291,1970,Other,Catholic,Female,1.0
1001079781,16330,36,15391,939,1966,White,Other,Male,1.0
1002394705,23145,80,19302,3843,1954,White,Jewish,Female,1.0
1003388590,20222,46,13748,6474,1963,Unknown,Pt Declined,Female,1.0
1005276406,19049,161,16513,2536,1966,Other,Unknown,Female,1.0
1009807096,17496,244,13718,3778,1968,Unknown,Other,Male,1.0
1009925127,27178,29,24699,2479,1944,Unknown,Catholic,Female,1.0
1012482111,21960,29,19385,2575,1958,White,Jewish,Female,1.0
1013701060,6083,35,4087,1996,1994,Hispanic/Latino,Unknown,Male,1.0


In [19]:
Med_schizo_COUNT_Control = get_has_certain_condition(Med_schizo_condition, Control_filtered_15_540, "Med_schizo", 180 , "COUNT", "Control")

Fetching data for Drug (...)


In [20]:
Med_schizo_COUNT_Control.loc[Med_schizo_COUNT_Control['number_of_occurences_Med_schizo'] != 0]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_Med_schizo
medical_record_number,,,,,,,,,
1000165222,17506,627,12215,5291,1970,Other,Catholic,Female,29.0
1001079781,16330,36,15391,939,1966,White,Other,Male,1.0
1002394705,23145,80,19302,3843,1954,White,Jewish,Female,11.0
1003388590,20222,46,13748,6474,1963,Unknown,Pt Declined,Female,3.0
1005276406,19049,161,16513,2536,1966,Other,Unknown,Female,2.0
1009807096,17496,244,13718,3778,1968,Unknown,Other,Male,5.0
1009925127,27178,29,24699,2479,1944,Unknown,Catholic,Female,1.0
1012482111,21960,29,19385,2575,1958,White,Jewish,Female,1.0
1013701060,6083,35,4087,1996,1994,Hispanic/Latino,Unknown,Male,28.0


In [21]:
Med_schizo_control = Med_schizo_EVER_Control.merge(Med_schizo_COUNT_Control['number_of_occurences_Med_schizo'], left_index = True, right_index = True)


In [22]:
Med_schizo_control

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_schizo,number_of_occurences_Med_schizo
medical_record_number,,,,,,,,,,
100002558,21414,15,19951,1463,1959,White,Other,Female,0.0,0.0
100002884,23929,44,18526,5403,1952,Unknown,,Male,0.0,0.0
1000068212,10116,29,9228,888,1990,White,Unknown,Female,0.0,0.0
1000083464,25630,20,23026,2604,1948,White,Catholic,Male,0.0,0.0
1000119948,17087,163,15757,1330,1971,African-American,None,Male,0.0,0.0
1000165222,17506,627,12215,5291,1970,Other,Catholic,Female,1.0,29.0
1000199143,26941,141,22805,4136,1944,White,Jewish,Female,0.0,0.0
1000205924,14867,108,10618,4249,1976,Unknown,Unknown,Female,0.0,0.0
1000212584,19220,15,16818,2402,1965,White,,Male,0.0,0.0


In [23]:
Med_schizo_control.to_parquet('Med_schizo_control.parquet')